# This notebook is used to explore differences in the `CLSim`-based simulation arrising from using `GEANT` or not

In order to run this, you have to have run the scripts in `../scripts/0_injection/`, `../scripts/2_clsim_prop/`, `../scripts/3_clsim_mcpe/`, `../scripts/4_clsim_prop_no_geant`, and `5_clsim_mcpe_no_geant`. If you do not want to run these scripts yourself, a potentially incomplete selection of files can be found by changin the `DATADIR` variable to `/data/user/jlazar/upgrade_simulation_check/data`. Let me know if you experience any issues.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
from upgrade_simulation_check.mcpe_variables import get_count_vars, count_hits_per_module, mcpe_timing
from upgrade_simulation_check.utils import is_gen1, is_upgrade, is_ref_degg

from plot_helpers import plot_nmcpe, plot_nmodules, nmcpe_histogram_ratio, nmodule_histogram_ratio

In [ ]:
from icecube import dataio, simclasses, dataclasses
from astropy.stats import bayesian_blocks
from matplotlib.gridspec import GridSpec
from scipy.stats import ks_2samp
from matplotlib import colors

In [ ]:
DATADIR = "../data/"

## First, we will look at the distributions of MCPEs and OMs that saw a MCPE

### Let's start by looking at the distribution of $N_{\mathrm{MCPE}}$ and $N_{\mathrm{chan}}$ on all OMs.

Now seems like an appropriate time to explain the `desc` variable.
The first number is the PDG encoding of the charged lepton; 11 is $e^{-}$, and 13 is $\mu^{-}$.
The second number is the energy of the lepton in GeV.
The six-character bit at the end is the cartesian offset of the injected interaction vertex from the reference module.
$p$ indicates that the offset is in the positive direction, while $n$ indicates the offset is in the negative direction.
For now, I am offsetting only in the positive $x$, positive $z$, and negative $z$ directions.

In [ ]:
particle = 13 # Should be in [11, 13]
energy = 5 # GeV
offset = 8 # m

desc = f"{particle}_{energy}_p{offset}p0p0"
file1 = f"{DATADIR}/clsim_mcpe/upgrade_checks_clsim_mcpe_{desc}.i3.zst"
file2 = f"{DATADIR}/clsim_mcpe_no_geant/upgrade_checks_clsim_mcpe_no_geant_{desc}.i3.zst"

In [ ]:
nmcpe1, nmodule1 = get_count_vars(file1)
nmcpe2, nmodule2 = get_count_vars(file2)

In [ ]:
plot_nmcpe(nmcpe1, nmcpe2, figname=f"../figures/nmcpe_scatter_all_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")
plot_nmodules(nmodule1, nmodule2, figname=f"../figures/nmodule_scatter_all_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")

In [ ]:
nmcpe_histogram_ratio(nmcpe1, nmcpe2, desc1="GEANT", desc2="no GEANT")
nmodule_histogram_ratio(nmodule1, nmodule2, nbins=7, desc1="GEANT", desc2="no GEANT")

### Now we do it for Gen1 OMs

In [ ]:
nmcpe1, nmodule1 = get_count_vars(file1, filter=is_gen1)
nmcpe2, nmodule2 = get_count_vars(file2, filter=is_gen1)

In [ ]:
plot_nmcpe(nmcpe1, nmcpe2, figname=f"../figures/nmcpe_scatter_gen1_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")
plot_nmodules(nmodule1, nmodule2, figname=f"../figures/nmodule_scatter_gen1_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")

In [ ]:
nmcpe_histogram_ratio(nmcpe1, nmcpe2, desc1="GEANT", desc2="no GEANT")
nmodule_histogram_ratio(nmodule1, nmodule2, nbins=7, desc1="GEANT", desc2="no GEANT")

### And once again, but now for only Upgrade OMs

In [ ]:
nmcpe1, nmodule1 = get_count_vars(file1, filter=is_upgrade)
nmcpe2, nmodule2 = get_count_vars(file2, filter=is_upgrade)

In [ ]:
plot_nmcpe(nmcpe1, nmcpe2, figname=f"../figures/nmcpe_scatter_upgrade_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")
plot_nmodules(nmodule1, nmodule2, figname=f"../figures/nmcodule_scatter_upgrade_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")

In [ ]:
nmcpe_histogram_ratio(nmcpe1, nmcpe2, desc1="GEANT", desc2="no GEANT")
nmodule_histogram_ratio(nmodule1, nmodule2, nbins=7, desc1="GEANT", desc2="no GEANT")

### And finally on the reference DEgg

In [ ]:
nmcpe1, nmodule1 = get_count_vars(file1, filter=is_ref_degg)
nmcpe2, nmodule2 = get_count_vars(file2, filter=is_ref_degg)

In [ ]:
plot_nmcpe(nmcpe1, nmcpe2, figname=f"../figures/nmcpe_scatter_degg_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")
plot_nmodules(nmodule1, nmodule2, figname=f"../figures/nmodule_scatter_degg_{desc}.pdf", xdesc="GEANT", ydesc="no GEANT")

In [ ]:
nmcpe_histogram_ratio(nmcpe1, nmcpe2, desc1="GEANT", desc2="no GEANT")
nmodule_histogram_ratio(nmodule1, nmodule2, nbins=7, desc1="GEANT", desc2="no GEANT")

## Let's try to quantify this further

In [ ]:
cmap_mcpe = plt.get_cmap("Purples_r")
cmap_module = plt.get_cmap("Oranges_r")

bounds = np.linspace(-2, 0, 21)
norm = colors.BoundaryNorm(bounds, cmap_mcpe.N)

In [ ]:
particle = 13

res = np.full((2, 9, 9), np.nan)
for idx, energy in enumerate(np.linspace(1, 9, 9, dtype=int)):
    for jdx, offset in enumerate(np.linspace(1, 9, 9, dtype=int)):
        desc = f"{particle}_{energy}_p{offset}p0p0"
        file1 = f"{DATADIR}/clsim_mcpe/upgrade_checks_clsim_mcpe_{desc}.i3.zst"
        file2 = f"{DATADIR}/clsim_mcpe_no_geant/upgrade_checks_clsim_mcpe_no_geant_{desc}.i3.zst"
        try:
            nmcpe1, nmodule1 = get_count_vars(file1)
            nmcpe2, nmodule2 = get_count_vars(file2)
        except RuntimeError:
            continue

        test = ks_2samp(nmcpe1, nmcpe2)
        res[0, idx, jdx] = test.pvalue
        test = ks_2samp(nmodule1, nmodule2)
        res[1, idx, jdx] = test.pvalue

#==== MCPE figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[0,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_mcpe,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("All OMs")

cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{MCPE}}\right)$")

plt.savefig(f"../figures/clsim_nmcpe_heatmap_{particle}_all.pdf")
plt.show()

#==== Module figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[1,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_module,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("All OMs")

cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{PMT}}\right)$")

plt.savefig(f"../figures/clsim_npmt_heatmap_{particle}_all.pdf")
plt.show()

In [ ]:
bins = np.logspace(-2, 0, 11)

h, _ = np.histogram(res[0, :, :].flatten(), bins=bins)

fig, ax = plt.subplots()
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{MCPE}}$")

h, _ = np.histogram(res[1, :, :].flatten(), bins=bins)
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{PMT}}$")

ax.set_xlabel(r"$p$-value")
ax.set_ylabel(r"$\frac{\mathrm{d}N}{\mathrm{d}p}$")

ax.loglog()
ax.legend()

plt.show()

### Now we do it for Gen1 DOMs

In [ ]:
res = np.full((2, 9, 9), np.nan)
for idx, energy in enumerate(np.linspace(1, 9, 9, dtype=int)):
    for jdx, offset in enumerate(np.linspace(1, 9, 9, dtype=int)):
        desc = f"{particle}_{energy}_p{offset}p0p0"
        file1 = f"{DATADIR}/clsim_mcpe/upgrade_checks_clsim_mcpe_{desc}.i3.zst"
        file2 = f"{DATADIR}/clsim_mcpe_no_geant/upgrade_checks_clsim_mcpe_no_geant_{desc}.i3.zst"

        try:
            nmcpe1, nmodule1 = get_count_vars(file1, filter=is_gen1)
            nmcpe2, nmodule2 = get_count_vars(file2, filter=is_gen1)
        except RuntimeError:
            continue
        
        test = ks_2samp(nmcpe1, nmcpe2)
        res[0, idx, jdx] = test.pvalue
        test = ks_2samp(nmodule1, nmodule2)
        res[1, idx, jdx] = test.pvalue

#==== MCPE figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[0,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_mcpe,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("Gen1 DOMs")

cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{MCPE}}\right)$")

plt.savefig(f"../figures/clsim_nmcpe_heatmap_{particle}_gen1.pdf")
plt.show()

#==== Module figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[1,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_module,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("Gen1 DOMs")

cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{PMT}}\right)$")

plt.savefig(f"../figures/clsim_npmt_heatmap_{particle}_gen1.pdf")
plt.show()

In [ ]:
bins = np.logspace(-2, 0, 11)

h, _ = np.histogram(res[0, :, :].flatten(), bins=bins)

fig, ax = plt.subplots()
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{MCPE}}$")

h, _ = np.histogram(res[1, :, :].flatten(), bins=bins)
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{PMT}}$")

ax.set_xlabel(r"$p$-value")
ax.set_ylabel(r"$\frac{\mathrm{d}N}{\mathrm{d}p}$")

ax.loglog()
ax.legend()

plt.show()

### Now only for Upgrade OMs

In [ ]:
res = np.full((2, 9, 9), np.nan)
for idx, energy in enumerate(np.linspace(1, 9, 9, dtype=int)):
    for jdx, offset in enumerate(np.linspace(1, 9, 9, dtype=int)):
        desc = f"{particle}_{energy}_p{offset}p0p0"
        file1 = f"{DATADIR}/clsim_mcpe/upgrade_checks_clsim_mcpe_{desc}.i3.zst"
        file2 = f"{DATADIR}/clsim_mcpe_no_geant/upgrade_checks_clsim_mcpe_no_geant_{desc}.i3.zst"

        try:
            nmcpe1, nmodule1 = get_count_vars(file1, filter=is_upgrade)
            nmcpe2, nmodule2 = get_count_vars(file2, filter=is_upgrade)
        except RuntimeError:
            continue
        
        test = ks_2samp(nmcpe1, nmcpe2)
        res[0, idx, jdx] = test.pvalue
        test = ks_2samp(nmodule1, nmodule2)
        res[1, idx, jdx] = test.pvalue

#==== MCPE figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[0,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_mcpe,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("Upgrade OMs")
cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{MCPE}}\right)$")

plt.savefig(f"../figures/clsim_nmcpe_heatmap_{particle}_upgrade.pdf")
plt.show()

#==== Module figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[1,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_module,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("Upgrade OMs")

cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{PMT}}\right)$")

plt.savefig(f"../figures/clsim_npmt_heatmap_{particle}_upgrade.pdf")
plt.show()

In [ ]:
bins = np.logspace(-2, 0, 11)

h, _ = np.histogram(res[0, :, :].flatten(), bins=bins)

fig, ax = plt.subplots()
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{MCPE}}$")

h, _ = np.histogram(res[1, :, :].flatten(), bins=bins)
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{PMT}}$")

ax.set_xlabel(r"$p$-value")
ax.set_ylabel(r"$\frac{\mathrm{d}N}{\mathrm{d}p}$")

ax.loglog()
ax.legend()

plt.show()

### And finally for the reference DEgg

In [ ]:
res = np.full((2, 9, 9), np.nan)
for idx, energy in enumerate(np.linspace(1, 9, 9, dtype=int)):
    for jdx, offset in enumerate(np.linspace(1, 9, 9, dtype=int)):
        desc = f"{particle}_{energy}_p{offset}p0p0"
        file1 = f"{DATADIR}/clsim_mcpe/upgrade_checks_clsim_mcpe_{desc}.i3.zst"
        file2 = f"{DATADIR}/clsim_mcpe_no_geant/upgrade_checks_clsim_mcpe_no_geant_{desc}.i3.zst"

        try:
            nmcpe1, nmodule1 = get_count_vars(file1, filter=is_ref_degg)
            nmcpe2, nmodule2 = get_count_vars(file2, filter=is_ref_degg)
        except RuntimeError:
            continue
        
        test = ks_2samp(nmcpe1, nmcpe2)
        res[0, idx, jdx] = test.pvalue
        test = ks_2samp(nmodule1, nmodule2)
        res[1, idx, jdx] = test.pvalue
        

#==== MCPE figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[0,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_mcpe,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("Reference DEgg")

cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{MCPE}}\right)$")

plt.savefig(f"../figures/clsim_nmcpe_heatmap_{particle}_degg.pdf")
plt.show()

#==== Module figure ====#
fig, ax = plt.subplots()
im = ax.imshow(
    np.log10(res[1,:,:][::-1]),
    extent=[0.5, 9.5, 0.5, 9.5],
    cmap=cmap_module,
    norm=norm
)

ax.set_xlabel(r"$\mathrm{Horizontal\, offset}~\left[\mathrm{m}\right]$")
ax.set_ylabel(r"$E_{\ell}~\left[\mathrm{GeV}\right]$")

ax.set_title("Reference DEgg")

cbar = plt.colorbar(im, label=r"$\log_{10}\left(p_{\mathrm{PMT}}\right)$")

plt.savefig(f"../figures/clsim_npmt_heatmap_{particle}_degg.pdf")
plt.show()

In [ ]:
bins = np.logspace(-2, 0, 11)

h, _ = np.histogram(res[0, :, :].flatten(), bins=bins)

fig, ax = plt.subplots()
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{MCPE}}$")

h, _ = np.histogram(res[1, :, :].flatten(), bins=bins)
ax.step(bins[:-1], h / h.sum() / np.diff(bins), where="pre", label=r"$N_{\mathrm{PMT}}$")

ax.set_xlabel(r"$p$-value")
ax.set_ylabel(r"$\frac{\mathrm{d}N}{\mathrm{d}p}$")

ax.loglog()
ax.legend()

plt.show()

## We can now check the timing information of MCPEs

In [ ]:
particle = 13 # must be in [11, 13, "pi"]
energy = 5 # GeV
offset = 8 # m

desc = f"{particle}_{energy}_p{offset}p0p0"
file1 = f"../data/clsim_mcpe/upgrade_checks_clsim_mcpe_{desc}.i3.zst"
file2 = f"../data/clsim_mcpe_no_geant/upgrade_checks_clsim_mcpe_no_geant_{desc}.i3.zst"

timing1 = mcpe_timing(file1, filter=is_ref_degg)
timing2 = mcpe_timing(file2, filter=is_ref_degg)

In [ ]:
times = np.linspace(0, 70, 71)

nmcpemin = 1

timing_cdfs1 = []
for t in timing1:
    if len(t) < nmcpemin:
        continue
    timing_cdfs1.append(np.array([(t<x).sum() / len(t) for x in times]))

timing_cdfs2 = []
for t in timing2:
    if len(t) < nmcpemin:
        continue
    timing_cdfs2.append(np.array([(t<x).sum() / len(t) for x in times]))

In [ ]:
fig = plt.figure(figsize=(12, 4))
gs = GridSpec(
    1, 2,
    wspace=0.05, hspace=0.05
)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])

for cdf in timing_cdfs1:
    ax0.plot(times, cdf, alpha=0.01, color="crimson")
ax0.plot(times, np.median(timing_cdfs1, axis=0), color="crimson", label="GEANT", lw=3)

for cdf in timing_cdfs2:
    ax1.plot(times, cdf, alpha=0.01, color="dodgerblue")
ax1.plot(times, np.median(timing_cdfs2, axis=0), color="dodgerblue", label="No GEANT", lw=3)

# Configure y-axes
ax0.set_label("CDF")
ax0.set_ylim(-0.05, 1.05)
ax1.set_ylim(-0.05, 1.05)
ax1.set_yticklabels([])

# Configure x-axes
ax0.set_xlim(0, 70)
ax1.set_xlim(0, 70)
ax0.set_xlabel(r"$t~\left[\mathrm{ns}\right]$")
ax1.set_xlabel(r"$t~\left[\mathrm{ns}\right]$")

ax0.legend(loc=2)
ax1.legend(loc=2)

plt.show()

In [ ]:
fig, ax = plt.subplots()

for cdf in timing_cdfs1:
    ax.plot(times, cdf, alpha=0.01, color="crimson")

for cdf in timing_cdfs2:
    ax.plot(times, cdf, alpha=0.01, color="dodgerblue")
    
ax.plot(times, np.median(timing_cdfs1, axis=0), color="crimson", label="GEANT", lw=3)
ax.plot(times, np.median(timing_cdfs2, axis=0), color="dodgerblue", label="No GEANT", lw=3)

# Configure y-axes
ax.set_label("CDF")
ax.set_ylim(-0.05, 1.05)

# Configure x-axes
ax.set_xlim(0, 70)
ax.set_xlabel(r"$t~\left[\mathrm{ns}\right]$")

ax.legend(loc=2)

plt.show()

## Let's see how this difference changes with the number of MCPEs

In [ ]:
times = np.linspace(0, 70, 71)
for nmcpemin in [1, 3, 5, 10, 30, 50]:

    #==== Compute ====#
    timing_cdfs1 = []
    for t in timing1:
        if len(t) < nmcpemin:
            continue
        timing_cdfs1.append(np.array([(t<x).sum() / len(t) for x in times]))
    
    timing_cdfs2 = []
    for t in timing2:
        if len(t) < nmcpemin:
            continue
        timing_cdfs2.append(np.array([(t<x).sum() / len(t) for x in times]))

    #==== Split plot ====#
    fig = plt.figure(figsize=(12, 4))
    gs = GridSpec(
        1, 2,
        wspace=0.05, hspace=0.05
    )

    ax0 = fig.add_subplot(gs[0])
    ax1 = fig.add_subplot(gs[1])
    
    for cdf in timing_cdfs1:
        ax0.plot(times, cdf, alpha=0.01, color="crimson")
    ax0.plot(times, np.median(timing_cdfs1, axis=0), color="crimson", label="GEANT", lw=3)
    
    for cdf in timing_cdfs2:
        ax1.plot(times, cdf, alpha=0.01, color="dodgerblue")
    ax1.plot(times, np.median(timing_cdfs2, axis=0), color="dodgerblue", label="No GEANT", lw=3)
    
    # Configure y-axes
    ax0.set_label("CDF")
    ax0.set_ylim(-0.05, 1.05)
    ax1.set_ylim(-0.05, 1.05)
    ax1.set_yticklabels([])
    
    # Configure x-axes
    ax0.set_xlim(0, 70)
    ax1.set_xlim(0, 70)
    ax0.set_xlabel(r"$t~\left[\mathrm{ns}\right]$")
    ax1.set_xlabel(r"$t~\left[\mathrm{ns}\right]$")

    # Configure legends
    ax0.legend(loc=2)
    ax1.legend(loc=2)
    
    # Configure text
    ax0.text(2, 0.87, r"$N_{\mathrm{MCPE}}\geq %d$" % nmcpemin)
    ax1.text(2, 0.87, r"$N_{\mathrm{MCPE}}\geq %d$" % nmcpemin)

    plt.savefig(f"../figures/geant_timing_comparisons_split_ncmpemin-{nmcpemin:03}_{desc}.pdf")
    
    plt.show()

    #==== Combined plot ====#
    fig, ax = plt.subplots()

    for cdf in timing_cdfs1:
        ax.plot(times, cdf, alpha=0.01, color="crimson")
    
    for cdf in timing_cdfs2:
        ax.plot(times, cdf, alpha=0.01, color="dodgerblue")
        
    ax.plot(times, np.median(timing_cdfs1, axis=0), color="crimson", label="GEANT", lw=3)
    ax.plot(times, np.median(timing_cdfs2, axis=0), color="dodgerblue", label="No GEANT", lw=3)
    
    # Configure y-axes
    ax.set_label("CDF")
    ax.set_ylim(-0.05, 1.05)
    
    # Configure x-axes
    ax.set_xlim(0, 70)
    ax.set_xlabel(r"$t~\left[\mathrm{ns}\right]$")
    
    ax.legend(loc=2)

    # Configure text
    ax.text(2, 0.83, r"$N_{\mathrm{MCPE}}\geq %d$" % nmcpemin)
    
    plt.savefig(f"../figures/geant_timing_comparisons_merged_ncmpemin-{nmcpemin:03}_{desc}.pdf")
    
    plt.show()